In [13]:
import random
import pandas as pd
from sage.graphs.graph_generators import graphs 


grafi = []  

def cikel(n):
    G = Graph()
    G.add_vertices(range(n))
    e = [(i, (i + 1) % n) for i in range(n)]
    G.add_edges(e)
    return G

def drevo(i):
    return [Graph(tree) for tree in graphs.trees(i)]
 
for i in range(3, 10):  
    grafi.append(cikel(i))
    grafi.extend(drevo(i))
    

grafi2 = [] 
for graf in grafi:
    if graf.diameter() >= 3:
        grafi2.append(graf)
    continue


###########################################################

def SDCTD_stevilo2(g):
    h = g.complement()

    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    p.set_objective(sum([x[v] for v in g]))

    for v in g.vertices():
        neighbors = g.neighbors(v)  
        if neighbors:
            p.add_constraint(sum((x[w] + x[v]) for w in neighbors) >= 1)
        else:
            p.add_constraint(x[v] == 1)
            
    zaustavi = False

    for v in g.vertices():
        neighbors_complement = h.neighbors(v)  
        if neighbors_complement:  
            p.add_constraint(sum(x[w] for w in neighbors_complement) >= 1)

        else: 
            zaustavi = True
            break 

    if not zaustavi == True:
        p.solve()
        x = p.get_values(x)
        return(sum(1 for i in x.values() if i == 1))
    else:
        return None
    
############################################################ 

def modularni_produkt(G, H):

    # Ustvari nov graf za modularni produkt
    MP = Graph()
    
    # Pridobi množici vozlišč grafov G in H
    V_G = G.vertices()
    V_H = H.vertices()
    
    # Ustvari množico vozlišč modularnega produkta kot kartezični produkt V(G) × V(H)
    V_MP = [(g, h) for g in V_G for h in V_H]
    MP.add_vertices(V_MP)
    
    # Dodaj povezave v modularni produkt
    for (g, h) in V_MP:
        for (g_prim, h_prim) in V_MP:
            if (g, h) != (g_prim, h_prim):  # Preveri, da ne gre za isto vozlišče
                # Preveri pogoje za povezavo
                if (g == g_prim and H.has_edge(h, h_prim)) or \
                   (h == h_prim and G.has_edge(g, g_prim)) or \
                   (G.has_edge(g, g_prim) and H.has_edge(h, h_prim)) or \
                   (not G.has_edge(g, g_prim) and not H.has_edge(h, h_prim)):
                    MP.add_edge((g, h), (g_prim, h_prim))
    
    return MP

#######################################################

rezultati = []

for m in range(1, 251):
    g1 = random.choice(grafi2)
    g2 = random.choice(grafi)
    # modularni produkt grafov g1 in g2
    g3 = modularni_produkt(g1, g2)

    # Izračun dominacijskega števila za vsak graf
    dominacijsko_stevilo_graf1 = SDCTD_stevilo2(g1)
    dominacijsko_stevilo_graf2 = SDCTD_stevilo2(g2)
    dominacijsko_stevilo_graf3 = SDCTD_stevilo2(g3)

    # Dodajanje rezultatov v seznam
    rezultati.append({
        'Izbira grafov': f'Izbira {m}',  # Tukaj smo popravili
        'SDCTD_graf1': dominacijsko_stevilo_graf1,
        'SDCTD_graf2': dominacijsko_stevilo_graf2,
        'SDCTD_modularni': dominacijsko_stevilo_graf3
    })

df = pd.DataFrame(rezultati)

# Definicija funkcij
def neenacba(row):
    # Preveri, če velja pogoj za produkt in kartezično vrednost
    if row['SDCTD_graf1'] + 2 >=  row['SDCTD_modularni']:
        return True
    return False

# Doda stolpca
df['neenačba'] = df.apply(neenacba, axis=1)

# Prešteje koliko False vrednosti je v tabeli
false_neenacba = (df['neenačba'] == False).sum()

print(f"Število False v stolpcu neenacba: {false_neenacba}")

print(df)


Število False v stolpcu neenacba: 0
    Izbira grafov SDCTD_graf1 SDCTD_graf2 SDCTD_modularni  neenačba
0        Izbira 1           3           4               4      True
1        Izbira 2           3           3               4      True
2        Izbira 3           3           3               4      True
3        Izbira 4           3           3               5      True
4        Izbira 5           3           3               5      True
..            ...         ...         ...             ...       ...
245    Izbira 246           4           3               4      True
246    Izbira 247           3           3               4      True
247    Izbira 248           2           3               4      True
248    Izbira 249           4           3               5      True
249    Izbira 250           4           3               4      True

[250 rows x 5 columns]
